# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

## Library Imports


In [2]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). 

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Import our IEX Cloud API token.

We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.


In [4]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [5]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()


In [6]:
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'A LR(BQNCGSKN/SG T LETAEMOEALSA)D',
 'calculationPrice': 'close',
 'open': 141.49,
 'openTime': 1657624528856,
 'openSource': 'cfaliiof',
 'close': 135.86,
 'closeTime': 1654613885175,
 'closeSource': 'ifafilco',
 'high': 139.56,
 'highTime': 1696319851341,
 'highSource': 'a myie 5ddn1erit cueple',
 'low': 134.03,
 'lowTime': 1670563224219,
 'lowSource': 'lm 1ap5ineyeddu ieertc ',
 'latestPrice': 140.97,
 'latestSource': 'Close',
 'latestTime': 'April 26, 2021',
 'latestUpdate': 1635404311592,
 'latestVolume': 67534099,
 'iexRealtimePrice': 138.39,
 'iexRealtimeSize': 25,
 'iexLastUpdated': 1638386100287,
 'delayedPrice': 136.14,
 'delayedPriceTime': 1680346720298,
 'oddLotDelayedPrice': 140.17,
 'oddLotDelayedPriceTime': 1694681451520,
 'extendedPrice': 136.2,
 'extendedChange': 0.18,
 'extendedChangePercent': 0.00136,
 'extendedPriceTime': 1676583690826,
 'previousClose': 134.82,
 'previousVolume': 81862069,
 'chang

In [7]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a DataFrame

The next thing we need to do is add our stock's price and market capitalization to a DataFrame.

In [8]:
columns = ['ticker', 'stock_price', 'market_cap', 'num_shares_to_buy']
df = pd.DataFrame(columns= columns)

In [9]:
df.append(
pd.Series([
    symbol,
    price,
    market_cap,
    'N/A'
],index= columns
),
    ignore_index= True)

,ticker,stock_price,market_cap,num_shares_to_buy
0,AAPL,140.97,2263026395002,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame.

In [10]:
df= pd.DataFrame(columns = columns)
for stock in stocks['Ticker'][:5]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    df = df.append(
        pd.Series(
        [stock,
        data['latestPrice'], data['marketCap'], 'N/A'],
        index = columns),
    ignore_index=True)

In [11]:
df

,ticker,stock_price,market_cap,num_shares_to_buy
0,A,140.07,43664933107,N/A
1,AAL,22.44,14469071376,N/A
2,AAP,204.73,13265285230,N/A
3,AAPL,139.81,2362218045448,N/A
4,ABBV,113.48,202341819268,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement!

In [12]:
def chunks(lst, n):
    for i in range(0, len(lst), n ):
        yield lst[i:i+n]

In [13]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

In [14]:
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [15]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
df= pd.DataFrame(columns = columns)
for symbol in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for sym in symbol.split(','):
        df = df.append(
        pd.Series(
        [
            sym,
            data[sym]['quote']['latestPrice'],
            data[sym]['quote']['marketCap'],
            'N/A'
        ], index = columns),ignore_index= True
        )
    
df

,ticker,stock_price,market_cap,num_shares_to_buy
0,A,143.24,42652610573,N/A
1,AAL,22.86,14195514751,N/A
2,AAP,206.91,13422786648,N/A
3,AAPL,135.59,2309193257203,N/A
4,ABBV,111.49,200399680632,N/A
...,...,...,...,...
500,YUM,120.25,35100860645,N/A
501,ZBH,180.59,36648954421,N/A
502,ZBRA,499.90,27142199654,N/A
503,ZION,54.67,9028703671,N/A


## Calculating the Number of Shares to Buy


In [16]:
portfolio_size = input('portfolio value: ')
try:
    val  =float(portfolio_size)
except ValueError:
    portfolio_size = input('Enter an integer: ')
    val  =float(portfolio_size)
    
    
    

portfolio value: 1000000


In [19]:
position_size = val/len(df.index)
for stock in range(0,len(df.index)):
    df.loc[stock, 'num_shares_to_buy'] = math.floor(position_size /df.loc[stock, 'stock_price'] )                             

In [20]:
df

,ticker,stock_price,market_cap,num_shares_to_buy
0,A,143.24,42652610573,13
1,AAL,22.86,14195514751,86
2,AAP,206.91,13422786648,9
3,AAPL,135.59,2309193257203,14
4,ABBV,111.49,200399680632,17
...,...,...,...,...
500,YUM,120.25,35100860645,16
501,ZBH,180.59,36648954421,10
502,ZBRA,499.90,27142199654,3
503,ZION,54.67,9028703671,36
